In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
# Load and prepare the data
df = pd.read_csv('archive/Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets_20240712.csv')


C:\Users\rvani\AppData\Local\Temp\ipykernel_16660\768425276.py:2: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('archive/Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets_20240712.csv')


In [3]:
df.columns

Index(['tbl', 'Year', 'quarter', 'citymarketid_1', 'citymarketid_2', 'city1',
       'city2', 'airportid_1', 'airportid_2', 'airport_1', 'airport_2',
       'nsmiles', 'passengers', 'fare', 'carrier_lg', 'large_ms', 'fare_lg',
       'carrier_low', 'lf_ms', 'fare_low', 'Geocoded_City1', 'Geocoded_City2',
       'tbl1apk'],
      dtype='object')

In [4]:
df = df[['city1', 'city2', 'airport_1', 'airport_2', 'nsmiles', 'passengers', 'fare', 'fare_low', 'fare_lg']]
df = df.dropna()  # Remove rows with missing values
routes = df.to_dict('records') 

In [5]:
df.columns

Index(['city1', 'city2', 'airport_1', 'airport_2', 'nsmiles', 'passengers',
       'fare', 'fare_low', 'fare_lg'],
      dtype='object')

In [6]:
# Parameters for the Genetic Algorithm
POPULATION_SIZE = 600
GENERATIONS = 1000
MUTATION_RATE = 0.2625

# Fitness function example (minimizes total fare and distance)
def fitness(schedule):
    total_fare = sum(flight['fare'] for flight in schedule)
    total_distance = sum(flight['nsmiles'] for flight in schedule)
    return -total_fare - 0.1 * total_distance  # Negative because we are minimizing

# Selection: Select parents based on fitness
def selection(population):
    # Calculate fitness scores
    weights = [fitness(individual) for individual in population]
    
    # Find the minimum fitness score
    min_fitness = min(weights)
    
    # Shift fitness scores to be positive by adding an offset if necessary
    if min_fitness <= 0:
        weights = [w - min_fitness + 1 for w in weights]
    
    # Select two parents based on adjusted fitness weights
    parents = random.choices(population, weights=weights, k=2)
    return parents


# Crossover: Combine two schedules to produce offspring
def crossover(parent1, parent2):
    cut_point = random.randint(1, len(parent1) - 1)
    child = parent1[:cut_point] + parent2[cut_point:]
    return child

# Mutation: Randomly adjust part of the schedule
def mutate(schedule):
    if random.random() < MUTATION_RATE:
        index = random.randint(0, len(schedule) - 1)
        # Mutation example: Change to a different random route
        schedule[index] = random.choice(routes)
    return schedule

In [7]:
# Generate initial population
def generate_initial_population():
    population = []
    for _ in range(POPULATION_SIZE):
        schedule = random.sample(routes, k=10)  # Randomly sample 10 routes
        population.append(schedule)
    return population

In [8]:
# Main Genetic Algorithm function
def genetic_algorithm():
    population = generate_initial_population()
    
    for generation in range(GENERATIONS):
        new_population = []
        
        # Selection, Crossover, and Mutation to create a new generation
        for _ in range(POPULATION_SIZE // 2):
            parent1, parent2 = selection(population)
            child1, child2 = crossover(parent1, parent2), crossover(parent2, parent1)
            new_population.extend([mutate(child1), mutate(child2)])
        
        population = new_population
        
        # Track the best solution
        best_schedule = max(population, key=fitness)
        print(f"Generation {generation}: Best Fitness = {fitness(best_schedule)}")
    
    return best_schedule

In [9]:
if __name__ == "__main__":
    best_schedule = genetic_algorithm()
    print("Optimized Schedule:", best_schedule)

Generation 0: Best Fitness = -2155.7799999999997
Generation 1: Best Fitness = -1927.7400000000002
Generation 2: Best Fitness = -2098.8500000000004
Generation 3: Best Fitness = -2049.7
Generation 4: Best Fitness = -1614.66
Generation 5: Best Fitness = -1694.28
Generation 6: Best Fitness = -1836.06
Generation 7: Best Fitness = -1736.48
Generation 8: Best Fitness = -1681.65
Generation 9: Best Fitness = -1747.13
Generation 10: Best Fitness = -1765.16
Generation 11: Best Fitness = -1760.26
Generation 12: Best Fitness = -1725.76
Generation 13: Best Fitness = -1567.69
Generation 14: Best Fitness = -1449.62
Generation 15: Best Fitness = -1556.0800000000002
Generation 16: Best Fitness = -1495.0100000000002
Generation 17: Best Fitness = -1440.77
Generation 18: Best Fitness = -1442.84
Generation 19: Best Fitness = -1492.42
Generation 20: Best Fitness = -1453.18
Generation 21: Best Fitness = -1350.64
Generation 22: Best Fitness = -1367.98
Generation 23: Best Fitness = -1224.88
Generation 24: Best 